#### Official Documentation: 

https://docs.databricks.com/aws/en/generative-ai/agent-framework/author-agent?language=Genie+Spaces#on-behalf-of-user-authentication

In [0]:
!pip install databricks-sdk==0.55.0 mlflow==3.1.0 databricks-agents uv --quiet
dbutils.library.restartPython()

# There are some python library dependency problems due to mlflow 3.1. version - ignore those or use 2.22.0 if you don't like red

In [0]:
from agent import agent
results = agent.predict({"messages": [{"role": "user", "content": "How many data points there are in Genie?"}]})

In [0]:
### Keep in mind this isn't proper streaming solution - just to demonstrate using batch messages instead of real streaming
for event in agent.predict_stream(
    {"messages": [{"role": "user", "content": "How many data points there are in Genie?"}]}
):
    print(event, "-----------\n")

In [0]:
# Determine Databricks resources to specify for automatic auth passthrough at deployment time
import mlflow
from mlflow.models.auth_policy import SystemAuthPolicy, UserAuthPolicy, AuthPolicy

# Here you can see which resources are supported currently
from mlflow.models.resources import (
  DatabricksVectorSearchIndex,
  DatabricksServingEndpoint,
  DatabricksSQLWarehouse,
  DatabricksFunction,
  DatabricksGenieSpace,
  DatabricksTable,
  DatabricksUCConnection
)

# Import to fetch model name from config.py file
from src.config import get_raw_configs


###############
# System Service Principal Auth
###############
resources = [DatabricksServingEndpoint(endpoint_name=get_raw_configs()['model_name'])]
system_auth_policy = SystemAuthPolicy(resources=resources)

###############
# On-behalf-of-user authentication. 
# Remember to specify the API scopes needed for the agent to access Databricks resources
###############

# Here you could add more api_scopes but do not add extra unnecessary permissions unless it's being used
user_auth_policy = UserAuthPolicy(
    api_scopes=[
        "dashboards.genie"
    ]
)

input_example = {
    "messages": [
        {
            "role": "user",
            "content": "How many data points there are in Genie?"
        }
    ]
}

with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(artifact_path='agent',
       #name="agent",
        python_model="agent.py",
        code_paths=["./src"],
        input_example=input_example,
        pip_requirements=[
            "mlflow==3.1.0",
            "databricks-sdk==0.55.0",
            "openai==1.69.0",
            "pydantic",
        ],
        auth_policy=AuthPolicy(
            system_auth_policy=system_auth_policy,
            user_auth_policy=user_auth_policy
        )
    )

In [0]:
mlflow.set_registry_uri("databricks-uc")

# TODO: define the catalog, schema, and model name for your UC model
catalog = ""
schema = ""
model_name = "genie_agent"
UC_MODEL_NAME = f"{catalog}.{schema}.{model_name}"

# register the model to UC
uc_registered_model_info = mlflow.register_model(
    model_uri=logged_agent_info.model_uri, name=UC_MODEL_NAME
)

In [0]:
from databricks import agents
agents.deploy(UC_MODEL_NAME, uc_registered_model_info.version, tags = {"env": "demo"})

## Validate model serving endpoint works after it has been deployed

In [0]:
from databricks.sdk import WorkspaceClient

messages = [{"role": "user", "content": "How many columns there are in Genie?"}]
endpoint = f"agents_{catalog}-{schema}-{model_name}"

w = WorkspaceClient()
client = w.serving_endpoints.get_open_ai_client()
response = client.chat.completions.create(model=endpoint, messages=messages)
print(response.messages[-1]['content'])

## And remember, vigirous testing and evaluation starts here!!! 

https://www.databricks.com/blog/introducing-enhanced-agent-evaluation

In [0]:
#your tests & evaluations. Databricks offers great out-of-the-box solutions as well.